## Rag (Method 1)

### Loading Model

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('NL_January2024.pdf')
docs=loader.load()

### Text Splitting

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=100)

In [3]:
chunks = text_splitter.split_documents(docs)

### Embedding And Storing

In [4]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [5]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embedding_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

c:\Users\Arjun\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Arjun\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
vector_db = Chroma.from_documents(documents=chunks,
                                  embedding=embedding_model,
                                  persist_directory="data_base_1",
                                  )


##### Load VectorDB (if have)

In [6]:
vector_db = Chroma(persist_directory="data_base_1",
                   embedding_function=embedding_model)

c:\Users\Arjun\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(
c:\Users\Arjun\anaconda3\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [7]:
retriver = vector_db.as_retriever(top_k=3)

### Testing Retriever

In [8]:
vector_db.similarity_search("where the Run for unity organized", k=3)

[Document(metadata={'page': 8, 'source': 'NL_January2024.pdf'}, page_content="9\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nRUN FOR UNITY\nDL, Jodhpur\nOn the Rashtriya Ekta Diwas \norganised on 31 October 2023, \nall\tthe\tofficers\t and\tstaff\tof\tthe\t\nDefence Laboratory, Jodhpur (DLJ) took the Rashtriya Ekta Diwas Pledge. Further, ‘Run for Unity’ was organised to spread the message of unity in society. The event was led by Shri RV Hara \nPrasad, Outstanding Scientist & \nDirector, DLJ. During his address, he highlighted the contributions made by the legend Sardar Vallabh Bhai Patel in the freedom struggle \nand national integration after \nindependence.\nISSA, Delhi \nOn 31 October 2023, Institute \nfor Systems Studies & Analyses \n(ISSA), Delhi, organised ‘Run for Unity’ from Metcalfe House to Qudasia Bagh, near ISBT \nKashmere Gate. It is an 18th-\ncentury garden complex with depictions of the Indian freedom struggle movement. \nAll\tofficers\t and\tstaff\tof\tthe\t\nlaboratory t

### Prompt Passing

In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model="mistral")


In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """ 
    Answer the following question based on the provided context.
    Note : analyse given context carefully
    <context>
    {context}
    </context>

    Question : {input}""")


In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [12]:
from langchain.chains import create_retrieval_chain
retrival_chain = create_retrieval_chain(retriver, document_chain)

In [13]:
ans = retrival_chain.invoke({"input" : "List of all the places where RUN FOR UNITY was organized"})

In [14]:
ans

{'input': 'List of all the places where RUN FOR UNITY was organized',
 'context': [Document(metadata={'page': 8, 'source': 'NL_January2024.pdf'}, page_content="9\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nRUN FOR UNITY\nDL, Jodhpur\nOn the Rashtriya Ekta Diwas \norganised on 31 October 2023, \nall\tthe\tofficers\t and\tstaff\tof\tthe\t\nDefence Laboratory, Jodhpur (DLJ) took the Rashtriya Ekta Diwas Pledge. Further, ‘Run for Unity’ was organised to spread the message of unity in society. The event was led by Shri RV Hara \nPrasad, Outstanding Scientist & \nDirector, DLJ. During his address, he highlighted the contributions made by the legend Sardar Vallabh Bhai Patel in the freedom struggle \nand national integration after \nindependence.\nISSA, Delhi \nOn 31 October 2023, Institute \nfor Systems Studies & Analyses \n(ISSA), Delhi, organised ‘Run for Unity’ from Metcalfe House to Qudasia Bagh, near ISBT \nKashmere Gate. It is an 18th-\ncentury garden complex with depictions of the India

In [15]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

'    * Defence Laboratory, Jodhpur (DLJ)  * ISSA, Delhi  * MTRDC, Bengaluru  * Naval Science & Technological Laboratory (NSTL), Visakhapatnam'

In [16]:
ans2 = retrival_chain.invoke({"input" : "List of all the places where Vigilance Awareness Week was organized"})

In [17]:
ans2

{'input': 'List of all the places where Vigilance Awareness Week was organized',
 'context': [Document(metadata={'page': 7, 'source': 'NL_January2024.pdf'}, page_content="8\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\nVIGILANCE AWARENESS WEEK\nVigilance Awareness Week is observed every year during the week in which the birthday of \nSardar Vallabhbhai Patel (31 October) falls. The theme of this year’s Vigilance Awareness Week was ‘Say No to Corruption, Commit to the Nation’.\nDL, Jodhpur\nVigilance Awareness Week was \ncelebrated during 30 October to 3 November 2023 at the Defence \nLaboratory, Jodhpur (DLJ). All \nthe\tofficers\tand\tstaff\ttook\ta\tpledge\t\nregarding vigilance. A lecture on 'Say No to Corruption: Commit \nto the Nation' was delivered by \nShri Harendra Kumar Mahawar, \nIPS, IG, Anti-Corruption Bureau, \nJodhpur, on the occasion.\nIRDE, Dehradun\nInstruments Research & \nDevelopment Establishment (IRDE), Dehradun, observed \nVigilance Awareness Week during \n30 October 

In [18]:
st2 = ans2['answer']
stt2 = st2.replace("\n", "  ")
stt2

'    The given text does not mention any details about "Vigilance Awareness Week" being organized. The text only mentions "Run for Unity" events being held on Rashtriya Ekta Diwas (October 31) at various DRDO laboratories and institutions, including ISSA, Delhi; DL, Jodhpur; NSTL, Visakhapatnam; and MTRDC, Bengaluru.'

In [19]:
ans = retrival_chain.invoke({"input" : "Give me the summary of KARNATAKA RAJYOTSAVA CELEBRATION"})

In [20]:
ans

{'input': 'Give me the summary of KARNATAKA RAJYOTSAVA CELEBRATION',
 'context': [Document(metadata={'page': 10, 'source': 'NL_January2024.pdf'}, page_content='11\n|   JANUARY 2024\nDRDO NEWSLETTEREVENTS\n(NPOL), Kochi celebrated its \n71st Annual Day on 15 December 2023. Dr S Unnikrishnan Nair, DS \n& Director, VSSC was the Chief \nGuest for the event. Dr R Rajesh, Sc ‘G’ & Chairman, NPOL Annual Day Organising Committee welcomed the gathering and presented an overview of the \nfunction. Dr Ajith Kumar K, OS \n& Director, NPOL presented the report on the achievements of the laboratory in both technical and non-technical areas. Shri \nWilson K Cherukulath, Sc ‘G’ & \nChairman, Works Committee & Secretary, Works Committee and General Secretaries of NPOL’s Civilian Employees union and NPOL’s R&D Employees union \nalso spoke on the occasion. Prizes \nwere distributed for the sports competitions held during the Annual Day. Instrumental music fusion by ‘RLV College of Music \nand Fine Arts’ 

In [21]:
st = ans['answer']
stt = st.replace("\n", "  ")
stt

'  The Microwave Tube Research & Development Centre (MTRDC), Bengaluru, celebrated Karnataka Rajyotsava on November 17th, 2023. Dr Ganesha Bhatt, Retired Principal of a MES educational institution, was the Chief Guest for the event. The gathering included the Director and Centre Head of LRDE & MTRDC, among others. The Chief Guest spoke about Kannada, a classical language with rich heritage and recipient of the highest number of Jnanapith awards next to Hindi. Cultural programs were organized and a flute maestro, Shri Srinidhi Katti, performed. At the end, Shri Jayateertha D proposed a vote of thanks.  In addition, an exercise was conducted between 10 Eng. Bde Secundrabad and 475 Eng. Bde Nasirabad at Chelak Nagraja Jaisalmer, Rajasthan from October 17-18, 2023. The aim of the exercise was to check the readiness of the Indian Army in responding to any nuclear eventuality. A team from DL, Jodhpur participated in the exercise and showcased various nuclear instruments and systems, includin